# FSB II. - Data cleaning
## What is this

## Installs

In [ ]:
! pip3 install convertbng
! pip3 install python-Levenshtein

     |████████████████████████████████| 14.0MB 326kB/s 
     |████████████████████████████████| 51kB 3.3MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149791 sha256=889190a26d3df6fe0512c774f4306beebce73625022fae383394e5f0f2eae1e6
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


## Imports

In [ ]:
# built-in modules
import re
import io
import requests
import warnings
import os
import sys

# numpy and pandas
import pandas as pd
import numpy as np

# helper module for pandas to be able to process excel spreadsheets - xlrd is obsolete
import openpyxl

# matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams

# widgets
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display

# plotly modules
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
liblist = sorted([e.name for e in os.scandir(r'/usr/local/lib/python'+sys.version[0:3]+'/dist-packages')])
[l for l in liblist if re.search(re.compile(".*bng.*", re.IGNORECASE), l) is not None]

['convertbng', 'convertbng-0.6.25.dist-info']

In [ ]:
liblist = sorted([e.name for e in os.scandir(r'/usr/local/lib/python'+sys.version[0:3]+'/dist-packages')])
[l for l in liblist if re.search(re.compile(".*leven.*", re.IGNORECASE), l) is not None]

['Levenshtein', 'python_Levenshtein-0.12.2.dist-info']

In [ ]:
# The elusive convertbng
from convertbng.util import convert_osgb36_to_lonlat
# Our lovely Levenstein distance module
import Levenshtein

## Module settings

In [ ]:
# changing default figure size and style for matplotlib
rcParams["figure.figsize"] = 20, 10
mpl.style.use('ggplot')
warnings.filterwarnings('ignore')
myBag = dict()

## Functions

In [ ]:
def throwaway_function_for_filter_elements(df):
  list_of_boroughs = sorted(df["ProperCase"].unique().tolist())

  list_of_years = df["CalYear"].unique().tolist()
  list_of_incidents = sorted(df["IncidentGroup"].unique().tolist())
  list_of_boroughs.insert(0, ".*") 
  list_of_incidents.insert(0, ".*") 

  year_selector = widgets.IntRangeSlider(
      description = "Year: ",
      min = df["CalYear"].unique().min(),
      max = df["CalYear"].unique().max(),
      step = 1,
      orientation='horizontal',
      readout=True,
      value = [df["CalYear"].unique().min(), df["CalYear"].unique().max()],
      style = {'description_width': 'initial'}
  )


  borough_selector = widgets.Dropdown(
      description = "Borough (ProperCase): ",
      value = ".*",
      options = [(i,i) for i in list_of_boroughs],
      continuous_update=True,
      style = {'description_width': 'initial'}
      )
  def get_available_wards():
      list_of_wards = []
      list_of_wards.insert(0, ".*")
      list_of_wards.extend(sorted(df[df["IncGeo_BoroughName"] == borough_selector.value.upper()]["IncGeo_WardNameNew"].unique().tolist()))
      return list_of_wards

  ward_selector = widgets.Dropdown(
      description = "Ward: ",
      value = ".*",
      options = [(i,i) for i in get_available_wards()],
      continuous_update=True,
      style = {'description_width': 'initial'}  
      )

  def on_value_change(change):
    ward_selector.options = [(i,i) for i in get_available_wards()]
    ward_selector.value = ".*"


  borough_selector.observe(on_value_change, names = "value")

  type_selector = widgets.Dropdown(
      description = "IncidentGroup: ",
      value = ".*",
      options = [(i,i) for i in list_of_incidents],
      style = {'description_width': 'initial'} )
  vbox = VBox([year_selector, type_selector,  borough_selector, ward_selector])
  def f(return_df = False):
    if return_df:
      return df
    return vbox
  return f


In [ ]:
def get_filtered_dataframe():
  filter_elements = get_filter_elements().children
  df = get_filter_elements(return_df = True)
  return df[(df.CalYear.isin(range(filter_elements[0].value[0], filter_elements[0].value[1]+1))) &
            (df["IncidentGroup"].str.match(re.compile(filter_elements[1].value), re.IGNORECASE)) &
             (df["ProperCase"].str.match(re.compile(filter_elements[2].value), re.IGNORECASE)) & 
             (df["IncGeo_WardNameNew"].str.match(re.compile(filter_elements[3].value), re.IGNORECASE))
            ]

## Data acquisition

In [ ]:
myBag["lfb_data_url"] = "https://data.london.gov.uk/download/london-fire-brigade-incident-records/b8f76a50-c7a0-4ff4-b3e4-7a42c5d0e8e3/LFB%20Incident%20data%20from%20January%202017.xlsx"

In [ ]:
faking_browser = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}
response = requests.get(myBag["lfb_data_url"], headers = faking_browser )
if(response.ok):
  print("Data has been retrieved.")
  bytes_data = response.content
  buffer = io.BytesIO(bytes_data)
  have_a_look_at_excel = pd.ExcelFile(buffer)
else:
  print("Eh-raw")

Data has been retrieved.


In [ ]:
myBag["df"] = df = pd.read_excel(have_a_look_at_excel)

## Narrowing the scope

In [ ]:
## Limiting the scope

In [ ]:
get_filter_elements = throwaway_function_for_filter_elements(myBag["df"])

In [ ]:
get_filter_elements()

## Filtering the dataframe

In [ ]:
myBag["last_filtered_df"] = get_filtered_dataframe()
myBag["last_filtered_df"].head(3)

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,UPRN,USRN,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,Latitude,Longitude,FRS,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpHoursRoundUp,Notional Cost (£)
0,000298-01012017,2017-01-01,2017,14:17:24,14,False Alarm,False alarm - Good intent,NaN,Non Residential,Restaurant/cafe,Correct incident location,E4 7QH,E4,10091774658,22869100,E09000031,WALTHAM FOREST,Waltham Forest,E05000593,CHINGFORD GREEN,CHINGFORD GREEN,539649.0,194764.0,539650,194750,51.634359,0.016468,London,Chingford,159.0,Chingford,296.0,Woodford,2.0,2.0,2.0,1.0,326.0
1,000051-01012017,2017-01-01,2017,00:50:06,0,Fire,Secondary Fire,NaN,Outdoor Structure,Small refuse/rubbish container,In street outside gazetteer location,E6 3EU,E6,46079106,22201202,E09000025,NEWHAM,Newham,E05000476,BOLEYN,BOLEYN,541703.0,182755.0,541750,182750,51.525937,0.041286,London,East Ham,262.0,Plaistow,NaN,NaN,1.0,1.0,1.0,1.0,326.0
2,000429-01012017,2017-01-01,2017,21:00:49,21,False Alarm,AFA,NaN,Non Residential,Pub/wine bar/bar,Correct incident location,SE15 4HY,SE15,200003394409,22501092,E09000028,SOUTHWARK,Southwark,E05011103,Goose Green,Goose Green,534131.0,175641.0,534150,175650,51.463855,-0.070499,London,Peckham,314.0,Peckham,NaN,NaN,1.0,1.0,1.0,1.0,326.0


## A quick reminder of data types and columns

In [ ]:
myBag["last_filtered_df"].describe()

,CalYear,HourOfCall,UPRN,USRN,Easting_m,Northing_m,Easting_rounded,Northing_rounded,Latitude,Longitude,FirstPumpArriving_AttendanceTime,SecondPumpArriving_AttendanceTime,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpHoursRoundUp,Notional Cost (£)
count,420243.000000,420243.000000,4.202430e+05,4.202430e+05,206742.000000,206742.000000,420243.000000,420243.000000,206742.000000,206742.000000,396059.000000,160962.000000,417989.000000,417989.000000,418405.000000,418280.000000,418280.000000
mean,2018.523906,13.469257,2.287758e+10,2.035041e+07,530644.419397,180399.331597,530670.990950,180522.519709,51.446808,-0.118710,310.205111,388.037332,1.383838,1.566948,1.620834,1.373597,460.732055
std,1.146158,6.247467,4.793035e+10,4.801780e+06,10344.461021,7484.717039,9733.341425,7395.221092,1.766109,0.149110,131.824625,147.289737,0.721471,0.874773,1.652881,6.886935,2294.186846
min,2017.000000,0.000000,0.000000e+00,4.200740e+06,503582.000000,155998.000000,503550.000000,155950.000000,0.000000,-0.510155,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,326.000000
25%,2018.000000,9.000000,0.000000e+00,2.040087e+07,525045.000000,175882.000000,525150.000000,176150.000000,51.467583,-0.199554,228.000000,294.000000,1.000000,1.000000,1.000000,1.000000,328.000000
50%,2019.000000,14.000000,0.000000e+00,2.120103e+07,530812.500000,181004.000000,530950.000000,181050.000000,51.513058,-0.116925,292.000000,363.000000,1.000000,1.000000,1.000000,1.000000,339.000000
75%,2020.000000,19.000000,1.002535e+10,2.210052e+07,536958.000000,185072.000000,536350.000000,185250.000000,51.549157,-0.026776,369.000000,452.000000,2.000000,2.000000,2.000000,1.000000,346.000000
max,2021.000000,23.000000,2.000044e+11,9.999042e+07,560251.000000,200899.000000,560250.000000,200850.000000,51.691568,0.310609,1200.000000,1200.000000,11.000000,14.000000,319.000000,2163.000000,709464.000000


In [ ]:
myBag["last_filtered_df"].dtypes

IncidentNumber                                    object
DateOfCall                                datetime64[ns]
CalYear                                            int64
TimeOfCall                                        object
HourOfCall                                         int64
IncidentGroup                                     object
StopCodeDescription                               object
SpecialServiceType                                object
PropertyCategory                                  object
PropertyType                                      object
AddressQualifier                                  object
Postcode_full                                     object
Postcode_district                                 object
UPRN                                               int64
USRN                                               int64
IncGeo_BoroughCode                                object
IncGeo_BoroughName                                object
ProperCase                     

## Dropping columns that are not needed #1

In [ ]:
columns_to_drop = "Postcode_full,Postcode_district,UPRN,USRN,Easting_m,Northing_m,FRS,"
columns_to_drop += "IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,"
columns_to_drop += "SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,"
columns_to_drop += "NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,Notional Cost (£)"
columns_to_drop = columns_to_drop.split(",")
myBag["last_filtered_df"].drop(columns = columns_to_drop, inplace = True)

In [ ]:
myBag["last_filtered_df"].columns

Index(['IncidentNumber', 'DateOfCall', 'CalYear', 'TimeOfCall', 'HourOfCall',
       'IncidentGroup', 'StopCodeDescription', 'SpecialServiceType',
       'PropertyCategory', 'PropertyType', 'AddressQualifier',
       'IncGeo_BoroughCode', 'IncGeo_BoroughName', 'ProperCase',
       'IncGeo_WardCode', 'IncGeo_WardName', 'IncGeo_WardNameNew',
       'Easting_rounded', 'Northing_rounded', 'Latitude', 'Longitude',
       'PumpHoursRoundUp'],
      dtype='object')

## Handling null values

In [ ]:
columns_with_null = myBag["last_filtered_df"].columns[myBag["last_filtered_df"].isna().sum() > 0].values
columns_with_null

array(['SpecialServiceType', 'Latitude', 'Longitude', 'PumpHoursRoundUp'],
      dtype=object)

In [ ]:
myBag["last_filtered_df"][columns_with_null].isna().sum()

SpecialServiceType    288444
Latitude              213501
Longitude             213501
PumpHoursRoundUp        1963
dtype: int64

In [ ]:
# handling textual nulls (object type)
missing_value_str = "missing_data" 
myBag["last_filtered_df"][columns_with_null[0]] = myBag["last_filtered_df"][columns_with_null[0]].fillna(missing_value_str)

In [ ]:
# handling numerical nulls
missing_value_num = 0
myBag["last_filtered_df"][columns_with_null[1:]] = myBag["last_filtered_df"][columns_with_null[1:]].fillna(missing_value_num)

In [ ]:
myBag["last_filtered_df"][columns_with_null].isna().sum()

SpecialServiceType    0
Latitude              0
Longitude             0
PumpHoursRoundUp      0
dtype: int64

## Standardising BNG values (easting and northing) to GPS coordinates

In [ ]:
bng_resolved = convert_osgb36_to_lonlat(myBag["last_filtered_df"]["Easting_rounded"] , myBag["last_filtered_df"]["Northing_rounded"])


In [ ]:
# Calculating difference between newly calulated and old values
myBag["last_filtered_df"]["Longitude_new"], myBag["last_filtered_df"]["Latitude_new"] = bng_resolved
myBag["last_filtered_df"]["lng_diff"], myBag["last_filtered_df"]["lat_diff"] =\
myBag["last_filtered_df"]["Longitude"] - myBag["last_filtered_df"]["Longitude_new"], \
myBag["last_filtered_df"]["Latitude"] - myBag["last_filtered_df"]["Latitude_new"]

In [ ]:
# Setting the difference to 0 where the latitude and longitude values were missing
myBag["last_filtered_df"]["lng_diff"][myBag["last_filtered_df"]["Longitude"] == 0] = 0
myBag["last_filtered_df"]["lat_diff"][myBag["last_filtered_df"]["Latitude"] == 0] = 0

#### Displaying the difference between the converted and original latitude and longitude values

In [ ]:
myBag["last_filtered_df"][["lng_diff", "lat_diff"]].describe()

,lng_diff,lat_diff
count,206742.000000,206742.000000
mean,-0.000002,-0.000002
std,0.000416,0.000259
min,-0.002579,-0.002683
25%,-0.000361,-0.000227
50%,-0.000001,-0.000001
75%,0.000356,0.000222
max,0.010164,0.002454


In [ ]:
myBag["last_filtered_df"][["IncidentNumber", "Longitude", "Latitude"]].head(3)

,IncidentNumber,Longitude,Latitude
0,000298-01012017,0.016468,51.634359
1,000051-01012017,0.041286,51.525937
2,000429-01012017,-0.070499,51.463855


## Breaking down the date column to its constituents

In [ ]:
def break_date_down(df, datecolumn = "DateOfCall"):
  df['Month'] = df[datecolumn].dt.month
  df['MonthStr'] = df[datecolumn].dt.month_name()

  df['DayOfWeek'] = df[datecolumn].dt.dayofweek + 1
  df['DayOfWeekStr'] = df[datecolumn].dt.day_name()

  df['DayOfMonth'] = df[datecolumn].dt.day
  df['DayOfYear'] = df[datecolumn].dt.dayofyear
  df['WeekOfMonth'] = df.DayOfMonth.apply(lambda d: (d - 1) // 7 + 1)
  df['WeekOfYear'] = df[datecolumn].dt.week

In [ ]:
break_date_down(myBag["last_filtered_df"])

In [ ]:
myBag["last_filtered_df"].head(3)

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_rounded,Northing_rounded,Latitude,Longitude,PumpHoursRoundUp,Longitude_new,Latitude_new,lng_diff,lat_diff,Month,MonthStr,DayOfWeek,DayOfWeekStr,DayOfMonth,DayOfYear,WeekOfMonth,WeekOfYear
0,000298-01012017,2017-01-01,2017,14:17:24,14,False Alarm,False alarm - Good intent,missing_data,Non Residential,Restaurant/cafe,Correct incident location,E09000031,WALTHAM FOREST,Waltham Forest,E05000593,CHINGFORD GREEN,CHINGFORD GREEN,539650,194750,51.634359,0.016468,1.0,0.016477,51.634233,-0.000009,0.000126,1,January,7,Sunday,1,1,1,52
1,000051-01012017,2017-01-01,2017,00:50:06,0,Fire,Secondary Fire,missing_data,Outdoor Structure,Small refuse/rubbish container,In street outside gazetteer location,E09000025,NEWHAM,Newham,E05000476,BOLEYN,BOLEYN,541750,182750,51.525937,0.041286,1.0,0.041961,51.525880,-0.000675,0.000057,1,January,7,Sunday,1,1,1,52
2,000429-01012017,2017-01-01,2017,21:00:49,21,False Alarm,AFA,missing_data,Non Residential,Pub/wine bar/bar,Correct incident location,E09000028,SOUTHWARK,Southwark,E05011103,Goose Green,Goose Green,534150,175650,51.463855,-0.070499,1.0,-0.070222,51.463932,-0.000277,-0.000076,1,January,7,Sunday,1,1,1,52


In [ ]:
myBag["last_filtered_df"].describe()

,CalYear,HourOfCall,Easting_rounded,Northing_rounded,Latitude,Longitude,PumpHoursRoundUp,Longitude_new,Latitude_new,lng_diff,lat_diff,Month,DayOfWeek,DayOfMonth,DayOfYear,WeekOfMonth,WeekOfYear
count,420243.000000,420243.000000,420243.000000,420243.000000,420243.000000,420243.000000,420243.00000,420243.000000,420243.000000,420243.000000,4.202430e+05,420243.000000,420243.000000,420243.000000,420243.000000,420243.000000,420243.000000
mean,2018.523906,13.469257,530670.990950,180522.519709,25.309680,-0.058401,1.36718,-0.118467,51.508449,-0.000001,-8.890706e-07,6.518112,4.029916,15.714265,182.950226,2.698936,26.661943
std,1.146158,6.247467,9733.341425,7395.221092,25.749920,0.120250,6.87147,0.140354,0.066378,0.000292,1.819033e-04,3.427202,1.996717,8.827325,104.663041,1.271662,14.965223
min,2017.000000,0.000000,503550.000000,155950.000000,0.000000,-0.510155,0.00000,-0.510603,51.287661,-0.002579,-2.682881e-03,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2018.000000,9.000000,525150.000000,176150.000000,0.000000,-0.114471,1.00000,-0.197396,51.469474,0.000000,0.000000e+00,4.000000,2.000000,8.000000,93.000000,2.000000,14.000000
50%,2019.000000,14.000000,530950.000000,181050.000000,0.000000,0.000000,1.00000,-0.115098,51.513712,0.000000,0.000000e+00,7.000000,4.000000,16.000000,185.000000,3.000000,27.000000
75%,2020.000000,19.000000,536350.000000,185250.000000,51.512428,0.000000,1.00000,-0.035624,51.550930,0.000000,0.000000e+00,9.000000,6.000000,23.000000,272.000000,4.000000,39.000000
max,2021.000000,23.000000,560250.000000,200850.000000,51.691568,0.310609,2163.00000,0.310604,51.691188,0.010164,2.453953e-03,12.000000,7.000000,31.000000,366.000000,5.000000,53.000000


## Investigating duplicates

In [ ]:
myBag["last_filtered_df"]["IncidentNumber"].duplicated().sum()

0

## Investigating structural errors: checking for multiple representations of the same value

In [ ]:
myBag["last_filtered_df"].columns

Index(['IncidentNumber', 'DateOfCall', 'CalYear', 'TimeOfCall', 'HourOfCall',
       'IncidentGroup', 'StopCodeDescription', 'SpecialServiceType',
       'PropertyCategory', 'PropertyType', 'AddressQualifier',
       'IncGeo_BoroughCode', 'IncGeo_BoroughName', 'ProperCase',
       'IncGeo_WardCode', 'IncGeo_WardName', 'IncGeo_WardNameNew',
       'Easting_rounded', 'Northing_rounded', 'Latitude', 'Longitude',
       'PumpHoursRoundUp', 'Longitude_new', 'Latitude_new', 'lng_diff',
       'lat_diff', 'Month', 'MonthStr', 'DayOfWeek', 'DayOfWeekStr',
       'DayOfMonth', 'DayOfYear', 'WeekOfMonth', 'WeekOfYear'],
      dtype='object')

In [ ]:
columns_to_investigate = 'IncidentGroup,StopCodeDescription,IncGeo_BoroughName,ProperCase,'
columns_to_investigate += 'IncGeo_WardName,IncGeo_WardNameNew'
columns_to_investigate = columns_to_investigate.split(",")

In [ ]:
def get_suspicious_values(df, columns_to_investigate, min_distance = 1, max_distance = 5):
  dodgy_columns = {}
  for col in columns_to_investigate:
    dodgy_columns[col] = []
    unique_values = df[col].unique().tolist()
    while(len(unique_values)>2):
      current_unique = unique_values.pop()
      
      for other_unique in unique_values:
        distance = Levenshtein.distance(current_unique.strip(), other_unique.strip())
        if(distance >= min_distance and distance <= max_distance):
          dodgy_columns[col].append([current_unique, other_unique])
  return dodgy_columns

def find_nas(df, columns_to_investigate, na_strings = ["applicable", "not applicable",  "na", "n\.a", "missing", "nan"]):
  dodgy_columns = {}
  for col in columns_to_investigate:
    dodgy_columns[col] = []
    unique_values = df[col].unique().tolist()
    for current_unique in unique_values:
      for na in na_strings:
        result = re.search(re.compile(".*" + na.strip() + ".*", re.IGNORECASE), current_unique.strip())
        if(result is not None):
          dodgy_columns[col].append([current_unique, na])
  return dodgy_columns


### Examining possible duplicates

In [ ]:
get_suspicious_values(myBag["last_filtered_df"], columns_to_investigate, max_distance = 1)

{'IncGeo_BoroughName': [],
 'IncGeo_WardName': [['CRAYFORD', 'CRANFORD'],
  ['CHADWELL', 'SHADWELL'],
  ['CHEAP', 'CHEAM'],
  ['BROAD STREET', 'BREAD STREET'],
  ['SOUTHFIELD', 'SOUTHFIELDS'],
  ['ELTHAM NORTH', 'FELTHAM NORTH'],
  ['ELTHAM WEST', 'FELTHAM WEST'],
  ["ST. MARY'S", "ST. MARK'S"],
  ["QUEEN'S PARK", 'QUEENS PARK']],
 'IncGeo_WardNameNew': [['CRAYFORD', 'CRANFORD'],
  ['CHADWELL', 'SHADWELL'],
  ['CHEAP', 'CHEAM'],
  ['BROAD STREET', 'BREAD STREET'],
  ['SOUTHFIELD', 'SOUTHFIELDS'],
  ['ELTHAM NORTH', 'FELTHAM NORTH'],
  ['ELTHAM WEST', 'FELTHAM WEST'],
  ["ST. MARY'S", "ST. MARK'S"],
  ["QUEEN'S PARK", 'QUEENS PARK']],
 'IncidentGroup': [],
 'ProperCase': [],
 'StopCodeDescription': []}

### Investigating further the possible duplicates
The ones that might be problematic:
- 'CHEAP' and 'CHEAM'
- "QUEEN'S PARK" and 'QUEENS PARK'
- 'SOUTHFIELD', 'SOUTHFIELDS'


#### 'CHEAP' and 'CHEAM'

In [ ]:
myBag["last_filtered_df"]\
[(myBag["last_filtered_df"]["IncGeo_WardName"] == "CHEAP") | (myBag["last_filtered_df"]["IncGeo_WardName"] == "CHEAM") ]\
[["IncGeo_BoroughName", "IncGeo_WardName"]].head(3)

,IncGeo_BoroughName,IncGeo_WardName
167,SUTTON,CHEAM
577,SUTTON,CHEAM
773,SUTTON,CHEAM


In [ ]:
myBag["last_filtered_df"][myBag["last_filtered_df"]["IncGeo_WardName"] == "CHEAP"]["IncGeo_BoroughName"].unique()

array(['CITY OF LONDON'], dtype=object)

In [ ]:
myBag["last_filtered_df"][myBag["last_filtered_df"]["IncGeo_WardName"] == "CHEAM"]["IncGeo_BoroughName"].unique()

array(['SUTTON'], dtype=object)

In [ ]:
myBag["last_filtered_df"]\
[(myBag["last_filtered_df"]["IncGeo_WardName"] == "QUEEN'S PARK") | (myBag["last_filtered_df"]["IncGeo_WardName"] ==  'QUEENS PARK') ]\
[["IncGeo_BoroughName", "IncGeo_WardName"]].head(3)

,IncGeo_BoroughName,IncGeo_WardName
140,BRENT,QUEENS PARK
443,WESTMINSTER,QUEEN'S PARK
1052,WESTMINSTER,QUEEN'S PARK


#### "QUEEN'S PARK" and 'QUEENS PARK'

In [ ]:
myBag["last_filtered_df"][myBag["last_filtered_df"]["IncGeo_WardName"] == 'QUEENS PARK']["IncGeo_BoroughName"].unique()

array(['BRENT'], dtype=object)

In [ ]:
myBag["last_filtered_df"][myBag["last_filtered_df"]["IncGeo_WardName"] == "QUEEN'S PARK"]["IncGeo_BoroughName"].unique()

array(['WESTMINSTER'], dtype=object)

#### 'SOUTHFIELD', 'SOUTHFIELDS'

In [ ]:
myBag["last_filtered_df"]\
[(myBag["last_filtered_df"]["IncGeo_WardName"] == "SOUTHFIELD") | (myBag["last_filtered_df"]["IncGeo_WardName"] ==  'SOUTHFIELDS') ]\
[["IncGeo_BoroughName", "IncGeo_WardName"]].head(3)

,IncGeo_BoroughName,IncGeo_WardName
257,WANDSWORTH,SOUTHFIELDS
1327,WANDSWORTH,SOUTHFIELDS
2533,WANDSWORTH,SOUTHFIELDS


In [ ]:
myBag["last_filtered_df"][myBag["last_filtered_df"]["IncGeo_WardName"] == 'SOUTHFIELD']["IncGeo_BoroughName"].unique()

array(['EALING'], dtype=object)

In [ ]:
myBag["last_filtered_df"][myBag["last_filtered_df"]["IncGeo_WardName"] == "SOUTHFIELDS"]["IncGeo_BoroughName"].unique()

array(['WANDSWORTH'], dtype=object)

### Investigating "N.A"

In [ ]:
find_nas(myBag["last_filtered_df"], columns_to_investigate)

{'IncGeo_BoroughName': [],
 'IncGeo_WardName': [["ST. LEONARD'S", 'na'],
  ['CASTLE BAYNARD', 'na'],
  ['FROGNAL AND FITZJOHNS', 'na'],
  ['Hainault', 'na'],
  ['CANARY WHARF', 'na'],
  ['DUNDONALD', 'na'],
  ['BETHNAL GREEN', 'na'],
  ['SNARESBROOK', 'na'],
  ['Castle Baynard', 'na'],
  ['HAINAULT', 'na']],
 'IncGeo_WardNameNew': [["ST. LEONARD'S", 'na'],
  ['CASTLE BAYNARD', 'na'],
  ['FROGNAL AND FITZJOHNS', 'na'],
  ['Hainault', 'na'],
  ['CANARY WHARF', 'na'],
  ['DUNDONALD', 'na'],
  ['BETHNAL GREEN', 'na'],
  ['SNARESBROOK', 'na'],
  ['Castle Baynard', 'na'],
  ['HAINAULT', 'na']],
 'IncidentGroup': [],
 'ProperCase': [],
 'StopCodeDescription': []}

## Dropping columns that are not needed #2

In [ ]:
myBag["last_filtered_df"].head(3)

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_rounded,Northing_rounded,Latitude,Longitude,PumpHoursRoundUp,Longitude_new,Latitude_new,lng_diff,lat_diff,Month,MonthStr,DayOfWeek,DayOfWeekStr,DayOfMonth,DayOfYear,WeekOfMonth,WeekOfYear
0,000298-01012017,2017-01-01,2017,14:17:24,14,False Alarm,False alarm - Good intent,missing_data,Non Residential,Restaurant/cafe,Correct incident location,E09000031,WALTHAM FOREST,Waltham Forest,E05000593,CHINGFORD GREEN,CHINGFORD GREEN,539650,194750,51.634359,0.016468,1.0,0.016477,51.634233,-0.000009,0.000126,1,January,7,Sunday,1,1,1,52
1,000051-01012017,2017-01-01,2017,00:50:06,0,Fire,Secondary Fire,missing_data,Outdoor Structure,Small refuse/rubbish container,In street outside gazetteer location,E09000025,NEWHAM,Newham,E05000476,BOLEYN,BOLEYN,541750,182750,51.525937,0.041286,1.0,0.041961,51.525880,-0.000675,0.000057,1,January,7,Sunday,1,1,1,52
2,000429-01012017,2017-01-01,2017,21:00:49,21,False Alarm,AFA,missing_data,Non Residential,Pub/wine bar/bar,Correct incident location,E09000028,SOUTHWARK,Southwark,E05011103,Goose Green,Goose Green,534150,175650,51.463855,-0.070499,1.0,-0.070222,51.463932,-0.000277,-0.000076,1,January,7,Sunday,1,1,1,52


In [ ]:
myBag["last_filtered_df"].columns

Index(['IncidentNumber', 'DateOfCall', 'CalYear', 'TimeOfCall', 'HourOfCall',
       'IncidentGroup', 'StopCodeDescription', 'SpecialServiceType',
       'PropertyCategory', 'PropertyType', 'AddressQualifier',
       'IncGeo_BoroughCode', 'IncGeo_BoroughName', 'ProperCase',
       'IncGeo_WardCode', 'IncGeo_WardName', 'IncGeo_WardNameNew',
       'Easting_rounded', 'Northing_rounded', 'Latitude', 'Longitude',
       'PumpHoursRoundUp', 'Longitude_new', 'Latitude_new', 'lng_diff',
       'lat_diff', 'Month', 'MonthStr', 'DayOfWeek', 'DayOfWeekStr',
       'DayOfMonth', 'DayOfYear', 'WeekOfMonth', 'WeekOfYear'],
      dtype='object')

In [ ]:
columns_to_drop2 = "lat_diff,lng_diff,Latitude,Longitude,Easting_rounded,Northing_rounded".split(",")
myBag["last_filtered_df"].drop(columns = columns_to_drop2, inplace = True)

## Saving the dataframe

In [ ]:
text_input = widgets.Text(description = "Enter the file name:", value = "temp.csv", style = {"description_width": "initial"})
text_input2 = widgets.Text(description = "Enter the pickled file name:", value = "temp.pkl", style = {"description_width": "initial"})

text_input

Text(value='temp.csv', description='Enter the file name:', style=DescriptionStyle(description_width='initial')…

In [ ]:
myBag["last_filtered_df"].to_csv(text_input.value)

In [ ]:
text_input2

Text(value='temp.pkl', description='Enter the pickled file name:', style=DescriptionStyle(description_width='i…

In [ ]:
myBag["last_filtered_df"].to_pickle(text_input2.value)